# Анализ экономического развития стран и его связи с другими показателями (Этап 1)

## Цель и задачи проекта
**Цель**: Разработать автоматизированный pipeline для сбора, обработки и анализа данных Всемирного банка за период 1960 – 2024 годы и определить, существует ли корреляция между уровнем ВВП на душу населения и ряда выбранных социально-экономических показателей: прямые иностранные инвестиции, инфляция (дефлятор ВВП, годовой % роста), валовая норма охвата начальным образованием, рабочая сила с высшим образованием, уровень безработицы (общий), рождаемость (на 1000 человек), доля населения, живущего за чертой бедности, общий внешний долг.

**Задачи данного этапа**:
- Загрузка через API ключевых данных: 
-- список кодов показателей,
-- список всех стран,
-- значения показателей;
- Проведение очистки и предобработки данных;
- Загрузка данных в базу данных.

## Содержание:
1. [Выгрузка данных с сайта Всемирного банка с помощью API](#soderjanie_1)
2. [Очистка и предобработка данных](#soderjanie_2)
3. [Создание соединения к базе данных](#soderjanie_3)
4. [Вывод](#soderjanie_4)


## 1. Выгрузка данных с сайта Всемирного банка спомощью API<a id='soderjanie_1'></a>

In [1]:
#Импортируем библиотеки
import requests
import json
import pandas as pd
from typing import List, Optional
import sqlalchemy

___

Получаем данные о странах

In [2]:
# Cохраняем API всемирного банка с информацией о странах 
url_countries = "https://api.worldbank.org/v2/country?format=json&per_page=500"  

In [3]:
try:
    # Отправляем GET запрос к World Bank API
    response_countries = requests.get(url_countries)
    # Проверяем, что запрос успешен (статус код 200)
    response_countries.raise_for_status() # Поднимает ошибку (4** или 5**) если статус некорректный

    # Десериализуем полученный JSON-ответ. 
    # Парсим ответ от API в вид JSON
    data_countries = response_countries.json()

# Обрабатываем ошибки, если такие возникнут
except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching data: {e}")
except (IndexError, KeyError, ValueError) as e:
    print(f"An error occurred while processing the data: {e}")


In [4]:
#Выделяем второй элемент в переменную страны
countries = data_countries[1]

In [5]:
#Создаем из полученных данных датафрейм
df_countries = pd.DataFrame(countries)

In [6]:
#Выводим первые 5 строк датафрейма
df_countries.head()

,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,"{'id': 'LCN', 'iso2code': 'ZJ', 'value': 'Lati...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'HIC', 'iso2code': 'XD', 'value': 'High...","{'id': 'LNX', 'iso2code': 'XX', 'value': 'Not ...",Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
2,AFG,AF,Afghanistan,"{'id': 'MEA', 'iso2code': 'ZQ', 'value': 'Midd...","{'id': 'MNA', 'iso2code': 'XQ', 'value': 'Midd...","{'id': 'LIC', 'iso2code': 'XM', 'value': 'Low ...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Kabul,69.1761,34.5228
3,AFR,A9,Africa,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
4,AFW,ZI,Africa Western and Central,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,


In [7]:
#Получим идентификатор региона из столбца регион: для этого вытаскиваем значения по ключу id
df_countries['region_id'] = df_countries.region.apply(lambda x: x.get('id', None))

In [8]:
#Получим информацию о регионах, административных регионах, уровню дохода
#и типу лендинга - вытаскиваем значения по ключу value
for col in ['region', 'adminregion', 'incomeLevel', 'lendingType']:
    df_countries[col + '_value'] = df_countries[col].apply(lambda cell: cell.get('value', None))

In [9]:
#Оставляем только нужные для анализа столбцы
df_countries = df_countries[['id', 'iso2Code', 'name', 'capitalCity', 'longitude', 'latitude',
         'region_id', 'region_value', 'adminregion_value', 'incomeLevel_value',
         'lendingType_value']]

In [10]:
#Выводим первые 5 строк датафрейма
df_countries.head()

,id,iso2Code,name,capitalCity,longitude,latitude,region_id,region_value,adminregion_value,incomeLevel_value,lendingType_value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,Latin America & Caribbean,,High income,Not classified
1,AFE,ZH,Africa Eastern and Southern,,,,NA,Aggregates,,Aggregates,Aggregates
2,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,MEA,"Middle East, North Africa, Afghanistan & Pakistan","Middle East, North Africa, Afghanistan & Pakis...",Low income,IDA
3,AFR,A9,Africa,,,,NA,Aggregates,,Aggregates,Aggregates
4,AFW,ZI,Africa Western and Central,,,,NA,Aggregates,,Aggregates,Aggregates


In [11]:
#Приводим названия столбцов к snake_case
old_cols = df_countries.columns.tolist()
new_cols = ['id', 'iso2_code', 'name', 'capital', 'longitude', 'latitude',
            'region_id', 'region', 'admin_region', 'income_level', 'lending_type']

change_dict = dict(zip(old_cols, new_cols))

In [12]:
df_countries = df_countries.rename(columns=change_dict)

In [13]:
#Меняем расположение столбцов
df_countries = df_countries[['id', 'iso2_code', 'name', 'region_id', 'region', 'admin_region',
         'income_level', 'capital', 'longitude', 'latitude', 'lending_type']]

In [14]:
#Выводим произвольные 3 строки датафрейма
df_countries.sample(3)

,id,iso2_code,name,region_id,region,admin_region,income_level,capital,longitude,latitude,lending_type
69,DEC,D7,Europe & Central Asia (IDA-eligible countries),NA,Aggregates,,Aggregates,,,,Aggregates
19,BEC,B7,Europe & Central Asia (IBRD-only countries),NA,Aggregates,,Aggregates,,,,Aggregates
215,PLW,PW,Palau,EAS,East Asia & Pacific,,High income,Koror,134.479,7.34194,IBRD


__________________

Получаем список индикаторов

In [15]:
# Cохраняем API всемирного банка с информацией об индикаторах
url_indicators = "https://api.worldbank.org/v2/indicators?format=json&per_page=30000"

In [16]:
try:
    # Отправляем GET запрос к World Bank API
    response_indicators = requests.get(url_indicators)
    # Проверяем, что запрос успешен (статус код 200)
    response_indicators.raise_for_status()   # Поднимает ошибку (4** или 5**) если статус некорректный

    # Десериализуем полученный JSON-ответ. 
    # Парсим ответ от API в вид JSON
    data_indicators = response_indicators.json()


# Обрабатываем ошибки, если такие возникнут
except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching data: {e}")
except (IndexError, KeyError, ValueError) as e:
    print(f"An error occurred while processing the data: {e}")

In [17]:
#Выделяем второй элемент в переменную индикатора
indicators=data_indicators[1]

In [18]:
#Используем метод json_normalize для нормализации JSON при создании датафрейма
df_indicators = pd.json_normalize(indicators)

In [19]:
#Выводим последние 5 строк датафрейма
df_indicators.tail(5)

,id,name,unit,sourceNote,sourceOrganization,topics,source.id,source.value
29307,ytil_some_dfcl_all,Youth idle rate (% of persons with some degree...,,,,[],92,Disability Data Hub (DDH)
29308,ytil_some_dfcl_fem,Youth idle rate (% of female persons with some...,,,,[],92,Disability Data Hub (DDH)
29309,ytil_some_dfcl_male,Youth idle rate (% of male persons with some d...,,,,[],92,Disability Data Hub (DDH)
29310,ytil_some_dfcl_rur,Youth idle rate (% of persons living in rural ...,,,,[],92,Disability Data Hub (DDH)
29311,ytil_some_dfcl_urb,Youth idle rate (% of persons living in urban ...,,,,[{}],92,Disability Data Hub (DDH)


In [20]:
#Приводим названия столбцов к snake_case
old_cols1 = df_indicators.columns.tolist()
new_cols1 = ['id',
 'name',
 'unit',
 'source_note',
 'source_organization',
 'topics',
 'source_id',
 'source_value']
change_dict1 = dict(zip(old_cols1, new_cols1))

In [21]:
df_indicators= df_indicators.rename(columns=change_dict1)

In [22]:
#Оставляем необходимые столбцы
df_indicators=df_indicators[['id',
 'name',
 'unit',
 'source_note',
 'source_organization',
 'source_id',
 'source_value']]

In [23]:
#Выводим произвольные 3 строки датафрейма
df_indicators.sample(3)

,id,name,unit,source_note,source_organization,source_id,source_value
13432,JI.EDU.17UP.FE,Average number of completed years in formal ed...,,,,86,Global Jobs Indicators Database (JOIN)
22200,SE.CLO.15Y.PRF.R.PS.2006.1,Above Proficiency;PISA 2006 for grade 15Y usin...,,,,12,Education Statistics
10462,fin37.39c.9,"Received government transfer: cash only, rural...",,,,28,Global Findex database


____


Получаем значения показателей

In [24]:
def worldbank_data(data_indicators: List[str],
                         start_year: int,
                         end_year: int,
                         language: str = 'en') -> pd.DataFrame:

    '''
    Получает данные показателей из API Всемирного банка.

    Args:
        data_indicators: Список кодов показателей (например, ['EN.GHG.CO2.IC.MT.CE.AR5'])
        start_year: Год начала периода
        end_year: Год окончания периода
        language: Язык данных ('en', 'ru' и т.д.)

    Returns:
        pandas.DataFrame с данными показателей
    '''

    base_url = "https://api.worldbank.org/v2/country/all/indicator" # endpoint API - адрес по которому мы обращаемся за данными


    # список для хранения данных о показателях
    all_data = []

    try:
        for data_indicator in data_indicators:
            # Формируем URL для запроса
            url_data = f"{base_url}/{data_indicator}"
            params = {
                'format': 'json',
                'date': f"{start_year}:{end_year}",
                'per_page': 20000  # Большое значение для получения всех данных
            }

            # Выполняем запрос к API
            response_data = requests.get(url_data, params=params)
            response_data.raise_for_status()

            data = response_data.json()

            # API возвращает массив, где первый элемент - метаданные, второй - данные
            if len(data) > 1 and isinstance(data[1], list):
                for item in data[1]:
                    if item.get('value') is not None:
                        all_data.append({
                            'country': item['country']['value'],
                            'country_code': item['countryiso3code'],
                            'indicator': item['indicator']['value'],
                            'indicator_code': item['indicator']['id'],
                            'year': int(item['date']),
                            'value': item['value']
                        })

        # Создаем DataFrame
        df = pd.DataFrame(all_data)

        if df.empty:
            print("Предупреждение: Не получено данных для указанных параметров")
            return df

        return df

    # Обрабатываем возможные ошибки при работе с API
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API: {e}")
        return pd.DataFrame()
    except (KeyError, IndexError, ValueError) as e:
        print(f"Ошибка при обработке данных: {e}")
        return pd.DataFrame()

In [25]:
# Получаем данные 
data_indicators = ['NY.GDP.MKTP.KD.ZG', 'NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD', 'SP.POP.TOTL', 'BX.KLT.DINV.CD.WD', 
              'NY.GDP.DEFL.KD.ZG', 'SE.PRM.ENRL', 'SL.TLF.ADVN.ZS', 'SL.UEM.TOTL.ZS', 'SP.DYN.CBRT.IN',
              'SI.POV.LMIC', 'GC.DOD.TOTL.GD.ZS'] #Показатели
start_year = 1960
end_year = 2024

df_data = worldbank_data(data_indicators, start_year, end_year)

#Выводим 10 произвольных строк датафрейма
df_data.sample(10)

,country,country_code,indicator,indicator_code,year,value
98035,Uzbekistan,UZB,"Primary education, pupils",SE.PRM.ENRL,1993,1.769300e+06
38208,Mali,MLI,GDP per capita (current US$),NY.GDP.PCAP.CD,2005,5.487304e+02
116858,Germany,DEU,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1987,1.120000e+01
91018,Cameroon,CMR,"Primary education, pupils",SE.PRM.ENRL,2009,3.350662e+06
32603,Belgium,BEL,GDP per capita (current US$),NY.GDP.PCAP.CD,1999,2.525280e+04
44060,Europe & Central Asia (IDA & IBRD countries),TEC,"Population, total",SP.POP.TOTL,1999,4.386503e+08
6510,Gabon,GAB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1985,-2.332949e+00
125289,United Kingdom,GBR,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,2004,1.190000e+01
8118,Japan,JPN,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1974,-1.225240e+00
4654,Cameroon,CMR,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,2013,4.995529e+00


___


## 2. Очистка и предобработка данных<a id='soderjanie_2'></a>

In [26]:
# Выводим первые строки датафрейма
df_countries.head()

,id,iso2_code,name,region_id,region,admin_region,income_level,capital,longitude,latitude,lending_type
0,ABW,AW,Aruba,LCN,Latin America & Caribbean,,High income,Oranjestad,-70.0167,12.5167,Not classified
1,AFE,ZH,Africa Eastern and Southern,NA,Aggregates,,Aggregates,,,,Aggregates
2,AFG,AF,Afghanistan,MEA,"Middle East, North Africa, Afghanistan & Pakistan","Middle East, North Africa, Afghanistan & Pakis...",Low income,Kabul,69.1761,34.5228,IDA
3,AFR,A9,Africa,NA,Aggregates,,Aggregates,,,,Aggregates
4,AFW,ZI,Africa Western and Central,NA,Aggregates,,Aggregates,,,,Aggregates


In [27]:
# Выводим информацию о датафрейме
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            296 non-null    object
 1   iso2_code     296 non-null    object
 2   name          296 non-null    object
 3   region_id     296 non-null    object
 4   region        296 non-null    object
 5   admin_region  296 non-null    object
 6   income_level  296 non-null    object
 7   capital       296 non-null    object
 8   longitude     296 non-null    object
 9   latitude      296 non-null    object
 10  lending_type  296 non-null    object
dtypes: object(11)
memory usage: 25.6+ KB


In [28]:
# Убираем строки с агрегированными значениями
df_countries=df_countries[df_countries['region']!='Aggregates']

In [29]:
# Выводим количество пропущенных строк в датафрейме
df_countries.isna().sum()

id              0
iso2_code       0
name            0
region_id       0
region          0
admin_region    0
income_level    0
capital         0
longitude       0
latitude        0
lending_type    0
dtype: int64

In [30]:
# Меняем тип данных на числовой
for column in ['longitude','latitude']:
    df_countries[column]=pd.to_numeric(df_countries[column])

In [31]:
# Выводим случайные 5 строк
df_countries.sample(5)

,id,iso2_code,name,region_id,region,admin_region,income_level,capital,longitude,latitude,lending_type
55,COD,CD,"Congo, Dem. Rep.",SSF,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Low income,Kinshasa,15.32220,-4.3250,IDA
49,CHL,CL,Chile,LCN,Latin America & Caribbean,,High income,Santiago,-70.64750,-33.4750,IBRD
279,URY,UY,Uruguay,LCN,Latin America & Caribbean,,High income,Montevideo,-56.06750,-34.8941,IBRD
160,LIE,LI,Liechtenstein,ECS,Europe & Central Asia,,High income,Vaduz,9.52148,47.1411,Not classified
252,SWZ,SZ,Eswatini,SSF,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Mbabane,31.46590,-26.5225,Blend


In [32]:
# Выводим информацию о датафрейме
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 295
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            217 non-null    object 
 1   iso2_code     217 non-null    object 
 2   name          217 non-null    object 
 3   region_id     217 non-null    object 
 4   region        217 non-null    object 
 5   admin_region  217 non-null    object 
 6   income_level  217 non-null    object 
 7   capital       217 non-null    object 
 8   longitude     211 non-null    float64
 9   latitude      211 non-null    float64
 10  lending_type  217 non-null    object 
dtypes: float64(2), object(9)
memory usage: 20.3+ KB


In [33]:
# Проверяем уникальные значения в категориальных столбцах
for column in ['region_id', 'region', 'income_level']:
    print(f'Уникальные значения в столбце {column}:')
    print(df_countries[column].sort_values().unique())
    print()

Уникальные значения в столбце region_id:
['EAS' 'ECS' 'LCN' 'MEA' 'NAC' 'SAS' 'SSF']

Уникальные значения в столбце region:
['East Asia & Pacific' 'Europe & Central Asia'
 'Latin America & Caribbean '
 'Middle East, North Africa, Afghanistan & Pakistan' 'North America'
 'South Asia' 'Sub-Saharan Africa ']

Уникальные значения в столбце income_level:
['High income' 'Low income' 'Lower middle income' 'Not classified'
 'Upper middle income']



In [34]:
for column in ['region_id', 'region', 'income_level']:
    print(f'Распределение данных по значениям столбца {column}:')
    print(df_countries[column].value_counts())
    print()

Распределение данных по значениям столбца region_id:
ECS    58
SSF    48
LCN    42
EAS    37
MEA    23
SAS     6
NAC     3
Name: region_id, dtype: int64

Распределение данных по значениям столбца region:
Europe & Central Asia                                58
Sub-Saharan Africa                                   48
Latin America & Caribbean                            42
East Asia & Pacific                                  37
Middle East, North Africa, Afghanistan & Pakistan    23
South Asia                                            6
North America                                         3
Name: region, dtype: int64

Распределение данных по значениям столбца income_level:
High income            86
Upper middle income    54
Lower middle income    50
Low income             25
Not classified          2
Name: income_level, dtype: int64



In [35]:
# Проверяем полные дубликаты в датафрейме
df_countries.duplicated().sum()

0

___

In [36]:
# Выводим случайные 5 строк датафрейма
df_indicators.sample(5)

,id,name,unit,source_note,source_organization,source_id,source_value
28244,UIS.NART.3.Q1.M.LPIA,"Total net attendance rate, upper secondary, po...",,The Adjusted Location Parity Index (LPIA) is c...,UNESCO Institute for Statistics,12,Education Statistics
23620,SE.CLO.8.LDSEV.M.TMS.2011.5,Learning Deprivation Severity;TIMSS 2011 for g...,,,,12,Education Statistics
19523,per_sa_os.bry_q5_rur,Beneficiary incidence in 5th quintile (richest...,,Percentage of program beneficiaries in a quint...,ASPIRE,29,The Atlas of Social Protection: Indicators of ...
3610,con28d.9,"Buys data package daily, rural (% age 15+)",,,,28,Global Findex database
17514,per_lm_alllm.adq_q1_urb,Adequacy of benefits in 1st quintile (poorest)...,,Total transfer amount received by all benefici...,ASPIRE,29,The Atlas of Social Protection: Indicators of ...


In [37]:
# Выводим информацию о датафрейме
df_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29312 entries, 0 to 29311
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   29312 non-null  object
 1   name                 29312 non-null  object
 2   unit                 29312 non-null  object
 3   source_note          29312 non-null  object
 4   source_organization  29312 non-null  object
 5   source_id            29312 non-null  object
 6   source_value         29312 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [38]:
# Выводим количество пропущенных строк в датафрейме
df_indicators.isna().sum()

id                     0
name                   0
unit                   0
source_note            0
source_organization    0
source_id              0
source_value           0
dtype: int64

In [39]:
# Меняем тип данных на числовой
df_indicators['source_id']=pd.to_numeric(df_indicators['source_id'])

In [40]:
# Выводим информацию о датафрейме
df_indicators.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29312 entries, 0 to 29311
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   29312 non-null  object
 1   name                 29312 non-null  object
 2   unit                 29312 non-null  object
 3   source_note          29312 non-null  object
 4   source_organization  29312 non-null  object
 5   source_id            29312 non-null  int64 
 6   source_value         29312 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.6+ MB


In [41]:
# Проверяем полные дубликаты в датафрейме
df_indicators.duplicated().sum()

0

____

In [42]:
# Выводим случайные 5 строк датафрейма
df_data.sample(5)

,country,country_code,indicator,indicator_code,year,value
17110,World,WLD,GDP (current US$),NY.GDP.MKTP.CD,1974,5.419514e+12
121760,Palau,PLW,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,2013,1.402400e+01
8631,Lesotho,LSO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,2006,4.230095e+00
130491,Switzerland,CHE,"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS,1995,2.249521e+01
69017,Mongolia,MNG,"Foreign direct investment, net inflows (BoP, c...",BX.KLT.DINV.CD.WD,2022,2.504295e+09


In [43]:
# Выводим информацию о датафрейме
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130744 entries, 0 to 130743
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   country         130744 non-null  object 
 1   country_code    130744 non-null  object 
 2   indicator       130744 non-null  object 
 3   indicator_code  130744 non-null  object 
 4   year            130744 non-null  int64  
 5   value           130744 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 6.0+ MB


In [44]:
# Выводим количество пропущенных строк в датафрейме
df_data.isna().sum()

country           0
country_code      0
indicator         0
indicator_code    0
year              0
value             0
dtype: int64

In [45]:
# Получаем список кодов стран
all_countries=df_countries['id'].tolist()

In [46]:
# Фильтруем датафрейм, удалив инфрмацию по агрегированным регионам, оставляя только строки по странам
df_data=df_data[df_data['country_code'].isin(all_countries)]

In [47]:
# Округлим значения до 2 знаков после запятой
df_data['value']=round(df_data['value'],2)

In [48]:
# Выводим информацию о датафрейме
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104893 entries, 2919 to 130743
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   country         104893 non-null  object 
 1   country_code    104893 non-null  object 
 2   indicator       104893 non-null  object 
 3   indicator_code  104893 non-null  object 
 4   year            104893 non-null  int64  
 5   value           104893 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 5.6+ MB


In [49]:
# Выводим случайные 5 строк
df_data.sample(5)

,country,country_code,indicator,indicator_code,year,value
19382,Colombia,COL,GDP (current US$),NY.GDP.MKTP.CD,1966,5.428519e+09
48539,Cambodia,KHM,"Population, total",SP.POP.TOTL,2005,1.343920e+07
92357,Fiji,FJI,"Primary education, pupils",SE.PRM.ENRL,1997,1.159650e+05
12263,Spain,ESP,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,1962,9.950000e+00
49779,Czechia,CZE,"Population, total",SP.POP.TOTL,2000,1.025506e+07


In [50]:
# Проверяем полные дубликаты в датафрейме
df_data.duplicated().sum()

0

В датафреймах 'df_countries', 'df_indicators', 'df_data' 
- типы данных скорректированы и соответствуют содержанию, 
- отсутствуют незаполненные поля,
- отсутствуют полные дубликаты,
- подозрительные категориальные значения отсутствуют.

_____

## 3. Создание соединения к базе данных<a id='soderjanie_3'></a>

In [51]:
# Параметры базы данных
USER ='postgres.tssandphkfjxkblfsuae'
PASSWORD = *******
HOST = 'aws-1-eu-central-1.pooler.supabase.com'
PORT = '6543'
DBNAME = 'postgres'

In [52]:
# Сформируем строку подключения к SQLAlchemy
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

In [53]:
# Создаем соединение
engine = sqlalchemy.create_engine(DATABASE_URL)

In [54]:
print(type(engine))

<class 'sqlalchemy.engine.base.Engine'>


In [55]:
# Tест соединения
try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!


Выгружаем полученные данные о странах, идентификаторах и их значениях в базу

In [56]:
df_countries.to_sql('countries', con=engine, if_exists='replace', index=False)

In [57]:
df_indicators.to_sql('indicators', con=engine, if_exists='replace', index=False)

In [58]:
df_data.to_sql('data', con=engine, if_exists='replace', index=False)

___

### Вывод<a id='soderjanie_4'></a>


На первом этапе реализации проекта проведена работа по изучению API Всемирного банка.

Разработан автоматизированный pipeline для сбора данных через API с сайта Всемирного банка за период 1960 – 2024 годы о кодах социально-экономических показателей, наименованию стран, значениях показателей, который формирует URL для запроса данных, обрабатывает пагинацию (данные возвращаются постранично), обрабатывает возможные ошибки, возвращает данные в формате JSON.
 
Также разработанный pipeline содержит раздел для очистки и предобработки данных Всемирного банка, парсинг JSON в  SQL.

В результате полученная с сайта Всемирного банка необходимая информация загружена в базу данных.